<a href="https://colab.research.google.com/github/ImpulsoGov/mensageria-mvp/blob/main/passo1_mvp_algoritmo_selecao_diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Seleção diária dos cidadãos para envio de mensagens

#### Configurações iniciais do ambiente

In [64]:
# autenticação
from google.colab import auth
auth.authenticate_user()

In [65]:
# conexão BQ
from google.cloud import bigquery
client = bigquery.Client(project='predictive-keep-314223')

In [66]:
# importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.oauth2 import service_account
from datetime import datetime

#### Consulta e preparação dos dados

In [67]:
# histórico e divisão dos grupos de teste e controle
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.divisao_teste_controle_equipes`
"""

query_job = client.query(query)
df = query_job.to_dataframe()

In [68]:
# dados atualizados para confirmação de pendência
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.unificado_lista_com_telefones_grupos_atendimentos`
"""

query_job = client.query(query)
df_pendencias = query_job.to_dataframe()

In [69]:
# identificação dos cidadãos com pendências em cito
df_pendencias['citopatologico_pendente_atual'] = df_pendencias['status_exame'].apply(lambda x: True if x in ('exame_nunca_realizado','exame_vencido','exame_vence_no_quadrimestre_atual') else False)

In [70]:
# função para identificar crônicos em pendência
def pendencia_cronicos(x):
    if (x['esta_na_lista_hipertensos'] == True and (x['realizou_afericao_ultimos_6_meses'] == False or x['realizou_consulta_hip_ultimos_6_meses'] == False)) or (x['esta_na_lista_diabeticos'] == True and (x['realizou_consulta_dia_ultimos_6_meses'] == False or x['realizou_solicitacao_hemoglobina_ultimos_6_meses'] == False)):
        return True
    else:
        return False

df_pendencias['cronicos_pendente_atual'] = df_pendencias.apply(pendencia_cronicos,axis=1)

In [71]:
# junção dos dados históricos com a verificação de pendência atual
df_unificado = df.merge(df_pendencias[['nome_do_paciente','data_de_nascimento','citopatologico_pendente_atual','cronicos_pendente_atual']],how='left', on=['nome_do_paciente','data_de_nascimento'])

In [72]:
# verificaçao se a linha de cuidado ainda está pendente
def pendencia_atualizada(x):
    if x['linha_cuidado']=='cronicos' and x['cronicos_pendente_atual']==True:
        return True
    elif x['linha_cuidado']=='citopatologico' and x['citopatologico_pendente_atual']==True:
        return True
    else:
        return False

df_unificado['pendencia_atualizada'] = df_unificado.apply(pendencia_atualizada,axis=1)

In [73]:
df_unificado = df_unificado[df_unificado['pendencia_atualizada']==True]
df_unificado = df_unificado.drop_duplicates()

In [74]:
# histórico de envios anteriores
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens`
"""

query_job = client.query(query)
df_historico_envio_mensagens = query_job.to_dataframe()

In [75]:
df_unificado['chave_cidadao'] = df_unificado['nome_do_paciente'].astype(str) + '_' + df_unificado['data_de_nascimento'].astype(str)
df_historico_envio_mensagens['chave_cidadao'] = df_historico_envio_mensagens['nome_do_paciente'].astype(str) + '_' + df_historico_envio_mensagens['data_de_nascimento'].astype(str)

# filtrando cidadãos que já receberam a mensagem
df_filtrado = df_unificado[~df_unificado['chave_cidadao'].isin(df_historico_envio_mensagens['chave_cidadao'])]

#filtrando casos com o celular preenchido incorreto
df_filtrado = df_filtrado[df_filtrado['celular_tratado']!=0]

#### Divisão por horários

In [76]:
# função para dividir os usuários em grupos de horários das mensagens
def dividir_grupos_equilibrado(df, num_grupos=3):
    def dividir_municipio(grupo):
        grupo_size = len(grupo)
        grupos = np.tile(range(1, num_grupos + 1), grupo_size // num_grupos + 1)[:grupo_size]
        np.random.shuffle(grupos)
        return grupos

    #considerando a divisão em equipes
    df['horario_grupo'] = df.groupby('equipe_ine')['equipe_ine'].transform(dividir_municipio)

    return df

df_dividido = dividir_grupos_equilibrado(df_filtrado)

In [77]:
# máximo de 15 pessoas por equipe, dia e linha de cuidado -> máximo de 5 pessoas por horário, equipe, dia e linha de cuidado
df_envio_diario = df_dividido.groupby(['municipio','equipe_ine','linha_cuidado','horario_grupo','grupo']).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)

<ipython-input-77-1753165cde40>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_envio_diario = df_dividido.groupby(['municipio','equipe_ine','linha_cuidado','horario_grupo','grupo']).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)


In [78]:
df_envio_dia_atual = df_envio_diario[['municipio', 'equipe_ine', 'equipe_nome', 'linha_cuidado','nome_do_paciente','data_de_nascimento','celular_tratado','horario_grupo','grupo']]
df_envio_dia_atual['data_envio'] = datetime.today().strftime('%Y-%m-%d')
df_envio_dia_atual['data_envio'] = df_envio_dia_atual['data_envio'].astype('datetime64[ns]')
df_envio_dia_atual['celular_tratado'] = df_envio_dia_atual['celular_tratado'].astype(str)
df_envio_dia_atual['horario_grupo'] = df_envio_dia_atual['horario_grupo'].astype(str)

<ipython-input-78-fa4e4aa4d33f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_envio_dia_atual['data_envio'] = datetime.today().strftime('%Y-%m-%d')
<ipython-input-78-fa4e4aa4d33f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_envio_dia_atual['data_envio'] = df_envio_dia_atual['data_envio'].astype('datetime64[ns]')
<ipython-input-78-fa4e4aa4d33f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [80]:
#adicionar dados na tabela de histórico
table_id = "predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens"
# incremento com os dados do dia atual
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(df_envio_dia_atual, table_id, job_config=job_config)
job.result()

LoadJob<project=predictive-keep-314223, location=US, id=2152ae12-34f9-4175-b3fa-52ca542c7828>